In [ ]:
import numpy as np
from numpy import sin, cos,exp,pi
from time import process_time, time
from scipy.optimize import curve_fit, least_squares
from scipy.fft import fft,fftshift,fft2, fftfreq, ifft
from scipy.signal import oaconvolve, medfilt2d

from scipy.ndimage import rotate
from scipy.interpolate import interp1d

import h5py
from skimage.transform import radon
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib import transforms
from PIL import Image, ImageFilter
import joblib as jb
import tifffile as tfl
import os
from IPython.display import clear_output
import re
import pandas as pd

In [ ]:
%run "C:\Users\gabri\Documents\Python Scripts\NPGMI-Library\NPGMI Base Code.ipynb"

### 2PGMI PhysRevA Data

In [ ]:
MONO_DATA = np.genfromtxt("mono_contrast_data.csv", delimiter = ",").T
MONO_ERROR = abs(MONO_DATA[1] - (np.genfromtxt("mono_error.csv", delimiter = ",").T)[1])
BI_DATA = np.genfromtxt("bichromatic_from_paper.csv", delimiter = ",").T
BI_ERROR = abs(BI_DATA[1] - (np.genfromtxt("bi_error_paper.csv", delimiter = ",").T)[1])
POLY_DATA = np.genfromtxt("polychromatic_from_paper.csv", delimiter = ",").T
POLY_ERROR = abs(POLY_DATA[1] - (np.genfromtxt("poly_error_paper.csv", delimiter = ",").T)[1])

### Simulations

In [ ]:
p = 2.4e-6
L1mono = 1.2
L1bi = 1.73
L1poly = 4.65

L0mono = -0.05
L0bi = 0.01
L0poly = 0.44
d0 = {"mono": -0.75e-3, "bi": +0.8e-3, "poly": -0.38e-3}


Lmono = 2.99 - L0mono
Lbi = 3.52 - L0bi
Lpoly = 8.8 - L0poly


polyspectrum = pd.read_excel("ng6_MCNP_spectrum.xlsx").to_numpy().T
polyspectrum[0]*=1e-9

lammono = np.array([4.4e-10])
plammono = np.array([1])

lambi = np.array([4.4e-10,2.2e-10])
plambi = np.array([3.2/4.2,1/4.2])

lampoly, plampoly = polyspectrum.copy()


mt = 5
x0 = np.array([0])

Nbc = 5e28*4.149e-15
h = 0.27*pi / (Nbc*4.4e-10)
hpoly = 0.2*pi / (Nbc*5e-10)

phimono = Nbc*lammono*h
phibi = Nbc*lambi*h

phipoly = Nbc*lampoly*hpoly


camsize = 2.5e-2
slitx = 230e-6
# slitxrange = np.linspace(180,300,13)*1e-6
slitxpoly = 590e-6
# slitxpolyrange = np.linspace(450,600,16)*1e-6


slity = 100e-6
xbin = 1e-6
ybin = 1e-6
res = 100e-6
respoly = 150e-6


G1sdmono = {"p":p, "L1": np.array([L1mono]), "phi":phimono}
G1sdmono["L2"] = np.linspace(7,16,19)*1e-3 + d0["mono"]


G2sdmono = {"p":p, "L1": G1sdmono["L1"] +  G1sdmono["L2"], "phi":phimono}
G2sdmono["L2"] = Lmono - (G1sdmono["L1"] + G1sdmono["L2"])

G1sdbi = {"p":p, "L1": np.array([L1bi]), "phi":phibi}
G1sdbi["L2"] = np.linspace(7,16,10)*1e-3 + d0["bi"]


G2sdbi = {"p":p, "L1": G1sdbi["L1"] +  G1sdbi["L2"], "phi":phibi}
G2sdbi["L2"] = Lbi - (G1sdbi["L1"] + G1sdbi["L2"])


G1sdpoly = {"p":p, "L1": np.array([L1poly]), "phi":phipoly}
G1sdpoly["L2"] = np.linspace(7,16,10)*1e-3 + d0["poly"]


G2sdpoly = {"p":p, "L1": G1sdpoly["L1"] +  G1sdpoly["L2"], "phi":phipoly}
G2sdpoly["L2"] = Lpoly - (G1sdpoly["L1"] + G1sdpoly["L2"])


# Ioff = np.linspace(-5000,5000,10)
simaptsmono = {"G1":RectGrating(G1sdmono),"G2":RectGrating(G2sdmono)}
siminitmono = {"lam":lammono, "plam": plammono, "L":Lmono,"d":G1sdmono["L2"],"mt":mt, "x": np.linspace(0,camsize,int(camsize/xbin)),"x0" : x0,"xbin":xbin,"ybin":ybin,\
           "pos":np.array([[0],[1]]) ,"camsize" : camsize ,"convmode" : "valid", "slitx": slitx, "slity" : slity, "res": res, "batches" : 2000}

simaptsbi = {"G1":RectGrating(G1sdbi),"G2":RectGrating(G2sdbi)}
siminitbi = {"lam":lambi, "plam": plambi, "L":Lbi,"d":G1sdbi["L2"],"mt":mt, "x": np.linspace(0,camsize,int(camsize/xbin)),"x0" : x0,"xbin":xbin,"ybin":ybin,\
           "pos":np.array([[0],[1]]) ,"camsize" : camsize ,"convmode" : "valid", "slitx": slitx, "slity" : slity, "res": res, "batches" : 2000}

simaptspoly = {"G1":RectGrating(G1sdpoly),"G2":RectGrating(G2sdpoly)}
siminitpoly = {"lam":lampoly, "plam": plampoly, "L":Lpoly,"d":G1sdpoly["L2"],"mt":mt, "x": np.linspace(0,camsize,int(camsize/xbin)),"x0" : x0,"xbin":xbin,"ybin":ybin,\
           "pos":np.array([[0],[1]]) ,"camsize" : camsize ,"convmode" : "valid", "slitx": slitxpoly, "slity" : slity, "res": respoly, "batches" : 2000}


In [ ]:
t0 = time()

simmono = PGMI(simaptsmono,siminitmono)
simmono.propagate_to("G1")
simmono.propagate_to("G2")
simmono.generate_after("G2")

simbi = PGMI(simaptsbi,siminitbi)
simbi.propagate_to("G1")
simbi.propagate_to("G2")
simbi.generate_after("G2")

simpoly = PGMI(simaptspoly,siminitpoly)
simpoly.propagate_to("G1")
simpoly.propagate_to("G2")
simpoly.generate_after("G2")

print(time() - t0, "seconds")


In [ ]:
monocont = simmono.get_value("contrast")
bicont = simbi.get_value("contrast")
polycont = simpoly.get_value("contrast")

monofitparams = simmono.get_value("fitparams")
bifitparams = simbi.get_value("fitparams")
polyfitparams = simpoly.get_value("fitparams")

### Plots

In [ ]:
fig, ax1 = plt.subplots(figsize  = (12,8))

colours = ["red","blue", "black"]

ax1.plot((simmono.d - d0["mono"])*1e3, monocont, color = colours[0], label = r"Monochromatic")
ax1.plot((simbi.d -d0["bi"])*1e3, bicont, color = colours[1], label = r"Bichromatic")

ax1.plot((simpoly.d-d0["poly"] )*1e3, polycont, color = colours[2], label = r"Polychromatic")

plt.gca().set_prop_cycle(None)

ax1.errorbar(MONO_DATA[0],MONO_DATA[1],MONO_ERROR, fmt = "ro", elinewidth = 0.5,capsize = 5)
ax1.errorbar(BI_DATA[0]*1e3,BI_DATA[1],BI_ERROR, fmt = "bo", elinewidth = 0.5,capsize = 5)
ax1.errorbar(POLY_DATA[0]*1e3,POLY_DATA[1],POLY_ERROR, fmt = "ko", elinewidth = 0.5,capsize = 5)

ax1.axvline((p**2/lammono )* 1e3, label = "Predicted max")

ax1.legend(fontsize = 20, loc = "upper right")
plt.xticks(fontsize = 20)
plt.yticks(fontsize = 20)

ax1.set_xlabel("Separation Distance $d$ [mm]", fontsize = 30)
ax1.set_ylabel("Contrast", fontsize = 30)

# ax1.set_title("Contrast vs. G1-G2 Separation, %d diffraction order%s" % (mt, "s" if mt > 1 else ""))
# fig.savefig("ContrastVsDfinal.png")
# ax1.set_xlim(0.55,9)
# plt.grid()
plt.show()
plt.close(fig)